In [0]:
# CV-DEMO1/azure_databricks/notebooks/00_init_paths.py

import sys
import os

print("Rozpoczynam inicjalizację środowiska Databricks dla monorepo 'CV-DEMO1'.")
print("-" * 70)

# --- Krok 1: Określenie DOKŁADNEJ ścieżki do głównego katalogu repozytorium (CV-DEMO1) ---
# Celem jest dodanie ścieżki do folderu 'CV-DEMO1' do sys.path.
# Będziemy szukać tego fragmentu w ścieżce zwróconej przez dbutils.

REPO_ROOT_DIR_NAME = 'CV-DEMO1' # Nazwa głównego katalogu repozytorium

try:
    # PREFEROWANA METODA DLA NOWSZYCH WERSJI DATABRICKS:
    full_notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    print(f"Pełna ścieżka bieżącego notebooka (dbutils): {full_notebook_path}")

    # Znajdź indeks, gdzie zaczyna się nazwa głównego repozytorium
    idx = full_notebook_path.find(REPO_ROOT_DIR_NAME)
    if idx != -1:
        # Utnij ścieżkę, aby uzyskać sam katalog repozytorium
        # Np. z '/Users/.../CV-DEMO1/azure_databricks/notebooks/my_notebook.py' -> '/Users/.../CV-DEMO1'
        calculated_project_root_path = full_notebook_path[:idx + len(REPO_ROOT_DIR_NAME)]
    else:
        # Awaryjne rozwiązanie: jeśli nazwa repo nie została znaleziona w ścieżce dbutils,
        # spróbuj z os.getcwd() i ręcznym cofaniem się.
        print(f"Ostrzeżenie: '{REPO_ROOT_DIR_NAME}' nie znaleziono w ścieżce notebooka. Próba użycia os.getcwd().")
        # Zakładamy, że os.getcwd() to .../azure_databricks/notebooks (lub common/notebooks)
        # BARDZO WAŻNE: Musisz dostosować liczbę '..' poniżej do faktycznej głębokości
        # notebooka względem katalogu CV-DEMO1.
        # Jeśli 00_init_paths.py jest w 'CV-DEMO1/azure_databricks/notebooks/', to 2x '..'
        # Jeśli 00_init_paths.py jest w 'CV-DEMO1/azure_databricks/common/notebooks/', to 3x '..'
        current_working_dir = os.getcwd()
        # Przyjmując, że jest w common/notebooks (jak pokazał Twój output):
        project_root_path_fallback = os.path.abspath(os.path.join(current_working_dir, '..', '..', '..'))
        calculated_project_root_path = project_root_path_fallback
        print(f"Obliczona ścieżka do korzenia projektu (fallback): {calculated_project_root_path}")


except NameError:
    # Awaryjne rozwiązanie dla środowisk bez dbutils (np. lokalne testy)
    print("Błąd: 'dbutils' nie znaleziono. Próba użycia os.getcwd().")
    current_working_dir = os.getcwd()
    # Zakładamy, że os.getcwd() to .../azure_databricks/common/notebooks
    # Potrzebujemy cofnąć się 3 razy do CV-DEMO1:
    project_root_path_fallback = os.path.abspath(os.path.join(current_working_dir, '..', '..', '..'))
    calculated_project_root_path = project_root_path_fallback
    print(f"Bieżący katalog notebooka (os.getcwd()): {current_working_dir}")
    print(f"Obliczona ścieżka do korzenia projektu (fallback): {calculated_project_root_path}")

print(f"Docelowa główna ścieżka repozytorium: {calculated_project_root_path}")


# --- Krok 2: Agresywne usuwanie WSZYSTKICH istniejących ścieżek z repozytorium ---
# Usuwamy wszystko, co zawiera 'CV-DEMO1' (ignorując wielkość liter),
# aby uniknąć konfliktów z automatem Databricks.
paths_to_remove = []
repo_identifier_lower = REPO_ROOT_DIR_NAME.lower()
for p in list(sys.path): # Iteruj po kopii sys.path
    if repo_identifier_lower in p.lower():
        paths_to_remove.append(p)

if paths_to_remove:
    print(f"\nUsuwam {len(paths_to_remove)} istniejących ścieżek repozytorium z sys.path:")
    for p_remove in reversed(paths_to_remove): # Usuwaj od końca
        if p_remove in sys.path:
            sys.path.remove(p_remove)
            print(f"  Usunięto: {p_remove}")
else:
    print("\nBrak istniejących ścieżek repozytorium do usunięcia.")


# --- Krok 3: Dodanie DOKŁADNEJ ścieżki głównego katalogu repozytorium na sam początek sys.path ---
# Zapewnia to, że nasza ścieżka ma najwyższy priorytet.
if calculated_project_root_path not in sys.path:
    sys.path.insert(0, calculated_project_root_path)
    print(f"\nDodano '{calculated_project_root_path}' na początek sys.path.")
else:
    print(f"\n'{calculated_project_root_path}' już istnieje na początku sys.path (nie dodano ponownie).")


# --- Krok 4: Opcjonalnie: Włącz autoreload (dla notebooków) ---
try:
    if 'autoreload' not in sys.modules:
        get_ipython().run_line_magic('load_ext', 'autoreload')
    get_ipython().run_line_magic('autoreload', '2')
    print("Włączono autoreload dla modułów Pythona.")
except NameError:
    print("Autoreload działa tylko w środowiskach IPython (np. Databricks Notebooks).")
except Exception as e:
    print(f"Wystąpił błąd podczas włączania autoreload: {e}")

# --- Krok 5: Inicjalizacja SparkSession (opcjonalnie) ---
from pyspark.sql import SparkSession

if 'spark' not in locals():
    spark = SparkSession.builder \
        .appName("CV Demo Project") \
        .getOrCreate()
    print("SparkSession zainicjalizowana.")
else:
    print("SparkSession już istnieje.")

print("\nKonfiguracja inicjalizacyjna zakończona.")
print("Aktualny sys.path po finalnej konfiguracji:")
for p in sys.path:
    print(p)